# Alpha Test Harness

Test ideas relating to alpha racing

Created by Michael George (AKA Logiqx)

Website: https://logiqx.github.io/gps-wizard/

In [1]:
import os

import numpy as np

from math import pi, cos, sqrt

from common_core import Printable, projdir, loadCsv

## Constants

In [2]:
EARTH_RADIUS = 6371009

METRES_PER_NM = 1852

ALPHA_MAX_DISTANCE = 500
ALPHA_MIN_DISTANCE = 200
ALPHA_PROXIMITY = 50

## Proximity Testing

Function to calculate the proximity between two nearby points.

See the 'haversine_vs_pythagoras' notebook for a full comparison of the Haversine Formula vs Pythagorean Theorem.

The code here is very similar to the aforementioned notebook but makes use of pre-calculated values where possible.

In [3]:
def estimateProximity(longitudes, y_offsets, x_scales, i1, i2):
    '''Estimate the Euclidean distance between two nearby points on a sphere using Pythagorean Theorem'''

    # Calculate distance north / south
    yDelta = y_offsets[i2] - y_offsets[i1]

    # For the sake of completeness we need to cope with the points either side of the 180th meridian
    longDelta = abs(longitudes[i2] - longitudes[i1])
    if longDelta > pi:
        longDelta -= 2 * pi

    # Calculate distance east / west
    xDelta = longDelta * x_scales[i2]

    # Apply Pythagorean theorem to determine the distance
    distance = sqrt(xDelta ** 2 + yDelta ** 2)

    return distance

## Process Track

In [4]:
def processTrack(filename):
    '''Process the track'''

    # Load the track and calculate the extra fields
    track = loadCsv(filename)
    extras = calculateExtras(track)
    
    # Process the alphas
    processAlphas(track, extras)

In [5]:
MIN_SPEED_FILTER = 2.5

def calculateExtras(track):
    '''Calculate all of the extra fields for the track. These fields can be calculated on a per-trackpoint basis.'''

    extras = {}

    # Convert timestamps into datetimes
    extras['datetime'] = np.datetime64('1989-12-31T00:00') + track['timestamp'].astype('timedelta64[s]')
    
    # Convert latitude and longitude values from semicircles to radians
    extras['latitude'] = np.radians(track['position_lat'] * (180.0 / 2 ** 31))
    extras['longitude'] = np.radians(track['position_long'] * (180.0 / 2 ** 31))

    # Calculate distance north / south of the equator (metres)
    extras['y_offset'] = extras['latitude'] * EARTH_RADIUS

    # Calculate the scaling factor for distances east / west of the primary meridian
    extras['x_scale'] = EARTH_RADIUS * np.cos(extras['latitude'])
    
    # Calculate cumulative distance (m) using speed (m/s)
    extras['total_distance'] = track['speed'].cumsum()
    
    # Apply speed filter (standard to GP3S)
    extras['filter'] = np.where(track['speed'] < MIN_SPEED_FILTER, 1, 0)
    
    return extras

In [6]:
def processAlphas(track, extras):
    '''Process the alphas'''

    # Avoid using the dictionaries being accessed repeatedly
    indices = track['idx']
    speeds = track['speed']
    cogs = track['cog']
    datetimes = extras['datetime']
    latitudes = extras['latitude']
    longitudes = extras['longitude']
    y_offsets = extras['y_offset']
    x_scales = extras['x_scale']
    filters = extras['filter']
    distances = extras['total_distance']
      
    # Look for all alphas
    best = 0
    proximityChecks = 0
    for i2 in range(len(track)):
        if filters[i2] == 0:
            alpha = 0
            skip = distances[i2] - ALPHA_MIN_DISTANCE
            cutoff = distances[i2] - ALPHA_MAX_DISTANCE
            i1 = i2 - 1

            # Skip stuff that is clearly not an alpha
            while i1 >= 0 and filters[i1] == 0 and distances[i1] > skip:
                i1 -= 1

            # Look for alpha
            while i1 >= 0 and filters[i1] == 0 and distances[i1] > cutoff:

                # Simple approach to compare two COG values
                cog_diff = 180 - abs(abs(cogs[i2] - cogs[i1]) - 180)              
                if cog_diff > 120:
                    proximity = estimateProximity(longitudes, y_offsets, x_scales, i1, i2)
                    proximityChecks += 1

                    if proximity <= 50:
                        speed = abs(distances[i2] - distances[i1]) / (i2 - i1) * 3600 / METRES_PER_NM
                        if speed > alpha:
                            alpha = speed

                i1 -= 1
                           
            if alpha > 14:
                if alpha > best:
                    best = alpha
                    
                #print('{} {} {:0.3f}'.format(indices[i2], datetimes[i2], alpha))

    print()
    print('BEST = {:0.3f} knots alpha'.format(best))
    print('{} proximity checks'.format(proximityChecks))

## Run Tests

In [7]:
if __name__ == '__main__':
    processTrack(os.path.join(projdir, 'sessions', '20220404', 'Speedsurfing20220404134130.csv'))


BEST = 22.014 knots alpha
38418 proximity checks
